# Import Module for computation & Authenticate GEE

In [1]:
import geemap, ee

In [2]:
ee.Initialize()

# Define area of interest for the computation of NDVI

In [9]:
# get our Nepal boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','New Zealand')).geometry()

# Define function for EVI computation

In [10]:
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

## Define the date for image

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

# Filter image collection and compute the NDVI for each image

In [12]:
Sentinel_data = ee.ImageCollection('COPERNICUS/S2').filterDate("2023-03-01","2023-03-31").filterBounds(aoi) \
    .map(getNDVI).map(addDate).median()

### Define color pallate for NDVI

In [13]:
color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}



### Visualize the result with follium 

In [14]:
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(Sentinel_data.clip(aoi).select('NDVI'), pallete, "NDVI")

map1.addLayerControl()
map1

Map(center=[-41.69398314807406, 172.87124491464206], controls=(WidgetControl(options=['position', 'transparent…